In [1]:
cd ..

/home/ec2-user/uclresearchanalysis


In [2]:
# Adding project path
import sys
sys.path.append('/home/ec2-user/uclresearchanalysis/configuration')

import builtins
# builtins.uclresearch_topic = 'GIVENCHY'
builtins.uclresearch_topic = 'HAWKING'
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'

from configuration import data_access
from configuration import config

dataAccessManager = data_access.DataAccess()

import json
import math
import numpy as np
from tqdm import tqdm
import datetime as dt
from datetime import timezone
from datetime import timedelta

('Reading config file from location: '
 '/home/ec2-user/uclresearchanalysis/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-13', '2018-03-14', '2018-03-15'],
          'eventname': "hawking's death",
          'phrases': ['stephen%20hawking%20death',
                      'stephen%20hawking%20die',
                      'stephen%20hawking%20passed%20away'],
          'starttime': 'Mar 13 12:00:00 -0500 2018'},
 'path': {'crawl': {'followers': '/home/ec2-user/uclresearchanalysis/followers',
                    'friends': '/home/ec2-user/uclresearchanalysis/friends'},
          'cwd': '/home/ec2-user/uclresearchanalysis/data/hawking',
          'ml': '/home/ec2-user/uclresearchanalysis/data/hawking/pickle',
          'networkx': {'all': '/home/ec2-user/uclresearchanalysis/data/hawking/pickle/networkx_all.dat',
                       'frien

In [3]:
unique_tweets = config.load_tweets_dataframe()
display(unique_tweets.head())

Loading data file from path /home/ec2-user/uclresearchanalysis/data/hawking/pickle/tweets.dat
'Loaded 112613 entires'


,user_id,user,reply_id,retweet_id,at_ids,text,time_lapsed,order,mention_and_reply
0,836922157,johnstempinNPR,NaN,NaN,[],BREAKING: We are aware of local reports in Lon...,0.00,1,[]
1,405808530,TheWebLender,NaN,NaN,[],Stephen Hawking has passed away \n#sad,1.58,2,[]
2,39582174,butterbob,NaN,836922157.0,[],RT @johnstempinNPR: BREAKING: We are aware of ...,1.78,3,[0]
3,2868539474,_Refused,NaN,NaN,[],Did Stephen Hawking just die?,2.97,4,[]
4,28820629,Rowaenthe,NaN,NaN,[],I am gutted to hear of Stephen Hawking's death...,3.10,5,[]


In [4]:
exposed_users = config.load_exposed_user_ids_selected()
infected_users = config.load_infected_user_ids()
all_ids = set(infected_users) | set(exposed_users)
print(len(all_ids))
userData = dataAccessManager.get_existing_db_users(list(all_ids))
userData.head()

Loading data file from path /home/ec2-user/uclresearchanalysis/data/hawking/pickle/exposed_user_ids_selected.dat
'Loaded 11262 entires'
Loading data file from path /home/ec2-user/uclresearchanalysis/data/hawking/pickle/infected_user_ids.dat
'Loaded 112613 entires'
123875


,contributors_enabled,created_at,description,favourites_count,followers_count,followers_json,friends_count,friends_json,geo_enabled,id,lang,listed_count,location,name,protected,screen_name,statuses_count,url,verified
0,0,2016-03-12 03:40:25,do not do mean things or else you will get pre...,4078,98,"[2389921309, 854836994723127297, 8993008969700...",206,"[3452152700, 914309345017499648, 2365089001, 2...",1,708497850850607104,en,1,,Jacobo Zacarías,0,hermano_jacobo,2534,None,0
1,0,2016-11-02 22:53:33,Love conspiracy theories and what goes with th...,108,36,"[950769443071668224, 996570805398982656, 87664...",74,"[351203222, 123542694, 876647009259724801, 512...",0,793949195728060417,en,1,"Avondale, AZ",The Squid,0,phdnctsquid,594,None,0
2,0,2017-06-10 01:12:26,"I may not know what my future looks like, but ...",22588,90,[],106,"[730734505045151744, 1604834112, 30914431, 603...",0,873347086988607490,en,0,"Fresno, CA",Potato,0,potato_regiment,1387,None,0
3,0,2016-08-13 02:04:03,"Wow elias, great moves. Keep it up. Proud of you",1418,45,"[3118935900, 1012152090955694080, 4679376744, ...",233,"[454237315, 222823005, 166747718, 2916458450, ...",0,764281331752632322,en,0,"California, USA",Elias Rafail,0,eraf24k,294,None,0
4,0,2017-04-09 16:27:53,Probably should change my terrible bio...,22989,225,[],992,"[975748546388520965, 29103639, 25917171, 21801...",0,851109421467828227,en,0,"Sydney, New South Wales",Max Lo,0,_8hahawhoooo,1177,None,0


In [5]:
df = unique_tweets.merge(userData, left_on='user_id', right_on='id', how='outer')

In [ ]:
def find_index_by_user_id(df, user_id):
    return df.user_id[df.user_id == user_id].index.tolist()[0]

def find_root_and_generation(df, index):
    row = df.iloc[index]
    time_lapsed = row.time_lapsed
    source_index = row.source_index
    generation = int(0)
    while source_index is not None:
        index = source_index
        row = df.iloc[index]
        source_index = row.source_index
        generation += 1
    root_time = row.time_lapsed
    return (index, generation, time_lapsed-root_time)

friends_not_found_list = []

df['source_candidates'] = [[] for _ in range(len(df))]
df['source_index'] = [None for _ in range(len(df))]
df['seed_index'] = [None for _ in range(len(df))]
df['generation'] = [None for _ in range(len(df))]
df['time_since_seed'] = [None for _ in range(len(df))]

for index in tqdm(range(len(df))):
    user_id = str(df.loc[index, 'user_id'])
    try:
        friends = set(json.loads(df.loc[index, 'friends_json'])) & infected_users
        friends_indexes = [find_index_by_user_id(df, x) for x in friends]
        try:
            friends_indexes.extend(df.loc[index, 'mention_and_reply'])
        except TypeError:
            pass
        friends_indexes = sorted([x for x in set(friends_indexes)])
        df.loc[index, 'source_candidates'].extend(friends_indexes)
        if (len(friends_indexes) > 0) and index < len(infected_users):
            if (friends_indexes[0] < index):
                df.loc[index, 'source_index'] = friends_indexes[0]
        df.loc[index, 'seed_index'], df.loc[index, 'generation'], df.loc[index, 'time_since_seed'] = find_root_and_generation(df, index)
    except KeyError:
        friends_not_found_list.append(index)

 46%|████▌     | 56630/123875 [29:48<35:23, 31.67it/s] 

In [ ]:
c1 = df['source_candidates'].apply(lambda x: len(x) == 0)
c2 = df['order'].apply(lambda x: math.isnan(x))
df = df.loc[~ (c1 & c2)]

In [ ]:
df = df.drop(['user_id', 'user', 'protected', 'retweet_id', 'reply_id', 'at_ids',
         'mention_and_reply', 'contributors_enabled',
        'url'], axis=1)

In [ ]:
start_timestamp = dt.datetime.strptime(config.settings['data']['starttime'], '%b %d %H:%M:%S %z %Y')
start_timestamp

def convert_utc_to_est(time_string):
    datetime_object = dt.datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S')
    return datetime_object.replace(tzinfo=timezone.utc).astimezone(tz=timezone(-timedelta(hours=5)))

def get_user_created_days(created_at):
    delta_time = start_timestamp - convert_utc_to_est(str(created_at))
    return max(delta_time.days, 0) + 1

In [ ]:
df['user_created_days'] = list(map(lambda created_at: get_user_created_days(created_at), df.created_at))

In [ ]:
df['normalized_statuses_count'] = np.divide(df.statuses_count, df.user_created_days)
df['normalized_followers_count'] = np.divide(df.followers_count, df.user_created_days)
df['normalized_favourites_count'] = np.divide(df.favourites_count, df.user_created_days)
df['normalized_listed_count'] = np.divide(df.listed_count, df.user_created_days)
df['normalized_friends_count'] = np.divide(df.friends_count, df.user_created_days)

In [ ]:
config.dump_users_dataframe(df)